In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error as mse
from tqdm import tqdm, tqdm_notebook
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

In [2]:
#path = 'E:\\competition\\ccf_car\\gongzhen'
path = ''
train_sales= pd.read_csv(path+'../ccf_car/train_sales_data.csv')#, engine='python')
train_search_data = pd.read_csv(path+'../ccf_car/train_search_data.csv')#, engine='python')
train_user_reply_data = pd.read_csv(path+'../ccf_car/train_user_reply_data.csv')#, engine='python')
test = pd.read_csv(path+'../ccf_car/evaluation_public.csv')#, engine='python')
#train_sales.drop(['province'],axis=1,inplace=True)
#train_search_data.drop(['province'],axis=1,inplace=True)
#test.drop(['province'],axis=1,inplace=True)

In [3]:
# train_sales\train_search_data\train_user_reply_data  拼接
data = pd.merge(train_sales, train_search_data, 'left', on=['adcode', 'model', 'regYear', 'regMonth'])
data = pd.merge(data, train_user_reply_data, 'left', on=['model', 'regYear', 'regMonth'])

In [4]:
# col, col2, col3 中 ，设1.5倍四分位距之外的数据为异常值，用上下四分位数的均值填充
col, col2, col3 = ['popularity', 'carCommentVolum', 'newsReplyVolum']
col_per = np.percentile(data[col],(25,75))
diff = 1.5*(col_per[1] - col_per[0])
col_per_in = (data[col] >= col_per[0] - diff) & (data[col] <= col_per[1] + diff)
 
col_per2 = np.percentile(data[col2],(25,75))
diff2 = 1.5*(col_per2[1] - col_per2[0])
col_per_in2 = (data[col2] >= col_per2[0] - diff2) & (data[col2] <= col_per2[1] + diff2)
 
col_per3 = np.percentile(data[col3],(25,75))
diff3 = 1.5*(col_per3[1] - col_per3[0])
col_per_in3 = (data[col3] >= col_per3[0] - diff3) & (data[col3] <= col_per3[1] + diff3)
 
data.loc[~col_per_in, col] = col_per.mean()
data.loc[~col_per_in2, col2] = col_per2.mean()
data.loc[~col_per_in3, col3] = col_per3.mean()

In [5]:
# 0.58973169000
def quantile_clip(group):
    #group.plot()
    group[group < group.quantile(.05)] = group.quantile(.05)
    group[group > group.quantile(.9)] = group.quantile(.9)
    #group.plot()
    #plt.show()
    return group
data['salesVolume'] = data.groupby(['adcode', 'model'])['salesVolume'].transform(quantile_clip)

In [6]:
# 统计销量
data['bt_ry_mean'] = data.groupby(['bodyType','regYear'])['salesVolume'].transform('mean')
data['ad_ry_mean'] = data.groupby(['adcode','regYear'])['salesVolume'].transform('mean')
data['md_ry_mean'] = data.groupby(['model','regYear'])['salesVolume'].transform('mean')

In [7]:
'''
一、lgb预测
'''
# 测试集并入
data = pd.concat([data, test], ignore_index=True)
data['label'] = data['salesVolume']
data['id'] = data['id'].fillna(0).astype(int)
del data['salesVolume'], data['forecastVolum']
# 填补测试集的车身类型
data['bodyType'] = data['model'].map(train_sales.drop_duplicates('model').set_index('model')['bodyType'])
# 编码 bodyType、model
for i in ['bodyType', 'model']:
    data[i] = data[i].map(dict(zip(data[i].unique(), range(data[i].nunique()))))
# 距离2016年的时间间隔，月数
data['mt'] = (data['regYear'] - 2016) * 12 + data['regMonth']
 
shift_feat = []
data['model_adcode'] = data['adcode'] + data['model']
data['model_adcode_mt'] = data['model_adcode'] * 100 + data['mt']

# 填充测试集特征值
for col in ['carCommentVolum','newsReplyVolum','popularity','bt_ry_mean','ad_ry_mean', 'md_ry_mean']:
    lgb_col_na = pd.isnull(data[col])
    data[col] = data[col].replace(0,1)
    data.loc[lgb_col_na,col] = \
    ((((data.loc[(data['regYear'].isin([2017]))&(data['regMonth'].isin([1,2,3,4])), col].values /
    data.loc[(data['regYear'].isin([2016]))&(data['regMonth'].isin([1,2,3,4])), col].values)))*
    data.loc[(data['regYear'].isin([2017]))&(data['regMonth'].isin([1,2,3,4])), col].values * 1.03).round()

# 每年的新年在第几月份
data['happyNY'] = 0
data.loc[(data['regYear'].isin([2016,2018])&data['regMonth'].isin([2])),'happyNY'] = 1
data.loc[(data['regYear'].isin([2017])&data['regMonth'].isin([1])),'happyNY'] = 1


# label 下移12个月，则测试集填充上了label
shift_nums=[4]
for i in shift_nums:
    shift_feat.append('shift_model_adcode_mt_label_{0}'.format(i))
    data['model_adcode_mt_{0}'.format(i)] = data['model_adcode_mt'] + i
    data_last = data[~data.label.isnull()].set_index('model_adcode_mt_{0}'.format(i))
    data['shift_model_adcode_mt_label_{0}'.format(i)] = data['model_adcode_mt'].map(data_last['label'])
window=[1,2,3,4,5,6]
m12_1    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==12) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1), 'salesVolume'].values
m12_2    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==12) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==2), 'salesVolume'].values
m12_3    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==12) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==3), 'salesVolume'].values
m12_4    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==12) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==4) , 'salesVolume'].values
m12_5    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==12) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==5) , 'salesVolume'].values
m12_6    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==12) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==6) , 'salesVolume'].values
m12_7    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==12) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==7) , 'salesVolume'].values

m12_1_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==1), 'salesVolume'].values * m12_1
m12_2_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==2), 'salesVolume'].values * m12_2
m12_3_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==3), 'salesVolume'].values * m12_3
m12_4_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==4) , 'salesVolume'].values * m12_4
m12_5_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==5) , 'salesVolume'].values * m12_5
m12_6_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==6) , 'salesVolume'].values * m12_6
m12_7_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==7) , 'salesVolume'].values * m12_7

data.loc[data.mt==4, 'shift_model_adcode_mt_label_4'] =  m12_1_volum/2 + m12_2_volum/4 + m12_3_volum/8 + \
                                                      m12_4_volum/16 + m12_5_volum/32 + m12_6_volum/64 + m12_7_volum/64

m12_1    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==11) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==12), 'salesVolume'].values
m12_2    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==11) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1), 'salesVolume'].values
m12_3    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==11) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==2), 'salesVolume'].values
m12_4    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==11) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==3) , 'salesVolume'].values
m12_5    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==11) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==4) , 'salesVolume'].values
m12_6    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==11) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==5) , 'salesVolume'].values
m12_7    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==11) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==6) , 'salesVolume'].values

m12_1_volum = data.loc[data.mt==4, 'shift_model_adcode_mt_label_4'].values * m12_1
m12_2_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==2), 'salesVolume'].values * m12_2
m12_3_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==3), 'salesVolume'].values * m12_3
m12_4_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==4) , 'salesVolume'].values * m12_4
m12_5_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==5) , 'salesVolume'].values * m12_5
m12_6_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==6) , 'salesVolume'].values * m12_6
m12_7_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==7) , 'salesVolume'].values * m12_7

data.loc[data.mt==3, 'shift_model_adcode_mt_label_4'] =  m12_1_volum/2 + m12_2_volum/4 + m12_3_volum/8 + \
                                                      m12_4_volum/16 + m12_5_volum/32 + m12_6_volum/64 + m12_7_volum/64

m12_1    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==10) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==11), 'salesVolume'].values
m12_2    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==10) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==12), 'salesVolume'].values
m12_3    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==10) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1), 'salesVolume'].values
m12_4    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==10) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==2) , 'salesVolume'].values
m12_5    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==10) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==3) , 'salesVolume'].values
m12_6    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==10) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==4) , 'salesVolume'].values
m12_7    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==10) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==5) , 'salesVolume'].values

m12_1_volum = data.loc[data.mt==3, 'shift_model_adcode_mt_label_4'].values * m12_1
m12_2_volum = data.loc[data.mt==4, 'shift_model_adcode_mt_label_4'].values * m12_2
m12_3_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==3), 'salesVolume'].values * m12_3
m12_4_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==4) , 'salesVolume'].values * m12_4
m12_5_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==5) , 'salesVolume'].values * m12_5
m12_6_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==6) , 'salesVolume'].values * m12_6
m12_7_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==7) , 'salesVolume'].values * m12_7

data.loc[data.mt==2, 'shift_model_adcode_mt_label_4'] =  m12_1_volum/2 + m12_2_volum/4 + m12_3_volum/8 + \
                                                      m12_4_volum/16 + m12_5_volum/32 + m12_6_volum/64 + m12_7_volum/64

m12_1    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==9) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==10), 'salesVolume'].values
m12_2    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==9) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==11), 'salesVolume'].values
m12_3    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==9) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==12), 'salesVolume'].values
m12_4    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==9) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==1) , 'salesVolume'].values
m12_5    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==9) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==2) , 'salesVolume'].values
m12_6    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==9) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==3) , 'salesVolume'].values
m12_7    = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==9) , 'salesVolume'].values / train_sales.loc[(train_sales.regYear==2017)&(train_sales.regMonth==4) , 'salesVolume'].values

m12_1_volum = data.loc[data.mt==2, 'shift_model_adcode_mt_label_4'].values * m12_1
m12_2_volum = data.loc[data.mt==3, 'shift_model_adcode_mt_label_4'].values * m12_2
m12_3_volum = data.loc[data.mt==4, 'shift_model_adcode_mt_label_4'].values * m12_3
m12_4_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==4) , 'salesVolume'].values * m12_4
m12_5_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==5) , 'salesVolume'].values * m12_5
m12_6_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==6) , 'salesVolume'].values * m12_6
m12_7_volum = train_sales.loc[(train_sales.regYear==2016)&(train_sales.regMonth==7) , 'salesVolume'].values * m12_7

data.loc[data.mt==1, 'shift_model_adcode_mt_label_4'] =  m12_1_volum/2 + m12_2_volum/4 + m12_3_volum/8 + \
                                                      m12_4_volum/16 + m12_5_volum/32 + m12_6_volum/64 + m12_7_volum/64

# 根据月份添加权重值
a = 6; b = 4
data['weightMonth'] = data['regMonth'].map({1:a, 2:a, 3:a, 4:a,
                                            5:b, 6:b, 7:b, 8:b, 9:b, 10:b, 11:b, 12:b,})

# 统一取log
data['n_carCommentVolum'] = np.log(data['carCommentVolum'])
data['n_popularity'] = np.log(data['popularity'])
data['n_newsReplyVolum'] = np.log(data['newsReplyVolum'])

print(data.mt.unique())

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28]


In [8]:
def score(data):
    pred = data.groupby(['adcode', 'model'])['pred_label'].agg(lambda x: list(x))
    label = data.groupby(['adcode', 'model'])['label'].agg(lambda x: list(x))
    label_mean = data.groupby(['adcode', 'model'])['label'].agg(lambda x: np.mean(x))
    data_agg = pd.DataFrame()
    data_agg['pred_label'] = pred
    data_agg['label'] = label
    data_agg['label_mean'] = label_mean
    nrmse_score = []
    for raw in data_agg.values:
        nrmse_score.append(mse(raw[0], raw[1]) ** 0.5 / raw[2])
    return 1 - np.mean(nrmse_score)

In [9]:
df_lgb = pd.DataFrame({'id': test['id']})
for col_add in ['ad_ry_mean', 'md_ry_mean', 'bt_ry_mean']:
    # 取用的字段，用于训练模型
    num_feat = shift_feat
    cate_feat = ['adcode', 'bodyType', 'model', 'regYear', 'regMonth', 'happyNY']
    features = num_feat + cate_feat + ['popularity', 'carCommentVolum', 'newsReplyVolum', 
                                       'n_popularity', 'n_carCommentVolum', 'n_newsReplyVolum', 'weightMonth'] + [col_add]  # [ad_ry_mean, md_ry_mean, bt_ry_mean]
 
    train_idx = (data['mt'] <= 20) # 小于等于20月以内的数据作为训练集
    valid_idx = (data['mt'].between(21, 24)) # 21到24个月的数据作为验证集
    test_idx = (data['mt'] > 24) # 大于24个月的是测试集
 
    # label
    data['n_label'] = np.log(data['label'])
 
    train_x = data[train_idx][features]
    train_y = data[train_idx]['n_label']
 
    valid_x = data[valid_idx][features]
    valid_y = data[valid_idx]['n_label']
    
    
    
 
    ############################### lgb ###################################
    lgb_model = lgb.LGBMRegressor(
        num_leaves=40, reg_alpha=1, reg_lambda=0.1, objective='mse',
        max_depth=-1, learning_rate=0.05, min_child_samples=5, random_state=2019,
        n_estimators=8000, subsample=0.8, colsample_bytree=0.8)
 
    lgb_model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)],
                  categorical_feature=cate_feat, early_stopping_rounds=100, verbose=300)
    data['pred_label'] = np.e ** lgb_model.predict(data[features])
    model = lgb_model
    # 特征重要程度
    print ('lgb特征重要程度：',sorted(dict(zip(train_x.columns,model.feature_importances_)).items(),key=lambda x: x[1], reverse=True))
    print('NRMSE的均值:',score(data = data[valid_idx]))
    model.n_estimators = model.best_iteration_
    model.fit(data[~test_idx][features], data[~test_idx]['n_label'], categorical_feature=cate_feat)
    data['forecastVolum'] = np.e ** model.predict(data[features])
    sub = data[test_idx][['id']]
    sub['forecastVolum'] = data[test_idx]['forecastVolum'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
    sub_lgb = sub.reset_index(drop=True)
    sub_lgb = sub_lgb[['id','forecastVolum']]
    print('lgb中forecastVolmn的0值数量：',(sub_lgb['forecastVolum']==0).sum())
    df_lgb[col_add] = sub_lgb['forecastVolum']

Training until validation scores don't improve for 100 rounds
[300]	valid_0's l2: 0.113983
[600]	valid_0's l2: 0.111103
[900]	valid_0's l2: 0.109772
[1200]	valid_0's l2: 0.108185
[1500]	valid_0's l2: 0.107045
[1800]	valid_0's l2: 0.106456
[2100]	valid_0's l2: 0.105942
[2400]	valid_0's l2: 0.105634
Early stopping, best iteration is:
[2373]	valid_0's l2: 0.105561
lgb特征重要程度： [('model', 16334), ('adcode', 11728), ('shift_model_adcode_mt_label_4', 11647), ('popularity', 10077), ('regMonth', 9658), ('carCommentVolum', 9112), ('newsReplyVolum', 8913), ('ad_ry_mean', 5056), ('n_popularity', 2071), ('n_newsReplyVolum', 1946), ('n_carCommentVolum', 1856), ('bodyType', 1495), ('regYear', 1399), ('happyNY', 972), ('weightMonth', 283)]
NRMSE的均值: 0.7384237387516983
lgb中forecastVolmn的0值数量： 0
Training until validation scores don't improve for 100 rounds
[300]	valid_0's l2: 0.116467
[600]	valid_0's l2: 0.113106
[900]	valid_0's l2: 0.111665
[1200]	valid_0's l2: 0.110511
[1500]	valid_0's l2: 0.109371
[18

In [ ]:
num_feat = shift_feat
cate_feat = ['adcode', 'bodyType', 'model', 'regYear', 'regMonth', 'happyNY']
features = num_feat + cate_feat + ['popularity', 'carCommentVolum', 'newsReplyVolum', 
                                    'n_popularity', 'n_carCommentVolum', 'n_newsReplyVolum', 'weightMonth'] + [col_add]  # [ad_ry_mean, md_ry_mean, bt_ry_mean]
 
train_idx = (data['mt'] <= 20) # 小于等于20月以内的数据作为训练集
valid_idx = (data['mt'].between(21, 24)) # 21到24个月的数据作为验证集
test_idx = (data['mt'] > 24) # 大于24个月的是测试集
 
# label
data['n_label'] = np.log(data['label'])
 
train_x = data[train_idx][features]
train_y = data[train_idx]['n_label']
 
valid_x = data[valid_idx][features]
valid_y = data[valid_idx]['n_label']
############################### lgb ###################################
lgb_model = lgb.LGBMRegressor(
    num_leaves=40, reg_alpha=1, reg_lambda=0.1, objective='mse',
    max_depth=-1, learning_rate=0.05, min_child_samples=5, random_state=2019,
    n_estimators=8000, subsample=0.8, colsample_bytree=0.8)
 
lgb_model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)],
                categorical_feature=cate_feat, early_stopping_rounds=100, verbose=300)
data['pred_label'] = np.e ** lgb_model.predict(data[features])
model = lgb_model
# 特征重要程度
print ('lgb特征重要程度：',sorted(dict(zip(train_x.columns,model.feature_importances_)).items(),key=lambda x: x[1], reverse=True))
print('NRMSE的均值:',score(data = data[valid_idx]))
model.n_estimators = model.best_iteration_
model.fit(data[~test_idx][features], data[~test_idx]['n_label'], categorical_feature=cate_feat)
data['forecastVolum'] = np.e ** model.predict(data[features])
sub = data[test_idx][['id']]
sub['forecastVolum'] = data[test_idx]['forecastVolum'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
sub_lgb = sub.reset_index(drop=True)
sub_lgb = sub_lgb[['id','forecastVolum']]
print('lgb中forecastVolmn的0值数量：',(sub_lgb['forecastVolum']==0).sum())
df_lgb[col_add] = sub_lgb['forecastVolum']

# hyperopt

In [59]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
import lightgbm as lgb
from sklearn.model_selection import train_test_split
#导入数据
data=pd.read_csv('../rst/rst_data.csv')
train_idx = (data['mt'] <= 24) # 小于等于20月以内的数据作为训练集
#valid_idx = (data['mt'].between(21, 24)) # 21到24个月的数据作为验证集
test_idx = (data['mt'] > 24) # 大于24个月的是测试集
train=data[train_idx]
test=data[test_idx]
features=['shift_model_adcode_mt_label_4', 'adcode', 'bodyType', 'model', 'regYear', 'regMonth', 'happyNY', 'province', 'popularity', 'carCommentVolum', 'newsReplyVolum', 'n_popularity', 'n_carCommentVolum', 'n_newsReplyVolum', 'weightMonth', 'bt_ry_mean']
#划分数据
X_train, X_test, y_train, y_test = train[features],test[features],train['label'],test['label']
train_data = lgb.Dataset(data=X_train,label=y_train)
test_data = lgb.Dataset(data=X_test,label=y_test)

In [64]:
from hyperopt import fmin, tpe, hp, partial

# 自定义hyperopt的参数空间
space = {
        "num_leaves": hp.randint("num_leaves",100),
        "max_depth": hp.randint("max_depth", 20),
        "min_child_samples": hp.randint("min_child_samples",10),
        "n_estimators": hp.quniform("n_estimators",5000,15000,11),
         "num_trees": hp.randint("num_trees", 300),
         'learning_rate': hp.uniform('learning_rate', 1e-3, 5e-1),
         "bagging_fraction": hp.randint("bagging_fraction", 5),
         "num_leaves": hp.randint("num_leaves", 6),
        'subsample': hp.uniform('subsample', 0.5, 0.99),
        'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 0.95),
         }

def argsDict_tranform(argsDict, isPrint=False):
    argsDict["num_leaves"] = argsDict["num_leaves"] + 5
    argsDict["max_depth"] = argsDict["max_depth"] + 5
    argsDict["min_child_samples"] = argsDict["min_child_samples"] + 1
    argsDict['num_trees'] = argsDict['num_trees'] + 150
    argsDict["learning_rate"] = argsDict["learning_rate"] * 0.02 + 0.05
    argsDict["bagging_fraction"] = argsDict["bagging_fraction"] * 0.1 + 0.5
    argsDict["num_leaves"] = argsDict["num_leaves"] * 3 + 10
    argsDict['subsample'] = argsDict['subsample'] * 0.02 + 0.05
    argsDict['colsample_bytree'] = argsDict['colsample_bytree'] * 0.02 + 0.05
    if isPrint:
        print(argsDict)
    else:
        pass

    return argsDict

In [65]:
from sklearn.metrics import mean_squared_error

def lightgbm_factory(argsDict):
    argsDict = argsDict_tranform(argsDict)

    params = {'nthread': -1,  # 进程数
              'num_leaves': argsDict['num_leaves'],
              'max_depth': argsDict['max_depth'],  # 最大深度
              'num_trees': argsDict['num_trees'],  # 树的数量
              "min_child_samples": argsDict["min_child_samples"],
              "learning_rate":argsDict["learning_rate"],
              'subsample':argsDict['subsample'],
              'colsample_bytree': argsDict['colsample_bytree'],
              'eta': argsDict['learning_rate'],  # 学习率
              'bagging_fraction': argsDict['bagging_fraction'],  # bagging采样数
              'num_leaves': argsDict['num_leaves'],  # 终点节点最小样本占比的和
              'objective': 'regression',
              'feature_fraction': 0.7,  # 样本列采样
              'lambda_l1': 0,  # L1 正则化
              'lambda_l2': 0,  # L2 正则化
              'bagging_seed': 100,  # 随机种子,light中默认为100
              }
    #rmse
    params['metric'] = ['rmse']

    model_lgb = lgb.train(params, train_data, num_boost_round=300, valid_sets=[test_data],early_stopping_rounds=100)

    return get_tranformer_score(model_lgb)

def get_tranformer_score(tranformer):

    model = tranformer
    prediction = model.predict(X_test, num_iteration=model.best_iteration)

    return mean_squared_error(y_test, prediction)

In [67]:
# 开始使用hyperopt进行自动调参
algo = partial(tpe.suggest, n_startup_jobs=10)
best = fmin(lightgbm_factory, space, algo=algo, max_evals=100, pass_expr_memo_ctrl=None)

  0%|                                                                                                   | 0/100 [00:00<?, ?it/s, best loss: ?]

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[1]	valid_0's rmse: 519.154                                                                                                                   
Training until validation scores don't improve for 100 rounds                                                                                 
[2]	valid_0's rmse: 502.971                                                                                                                   
[3]	valid_0's rmse: 484.374                                                                                                                   
[4]	valid_0's rmse: 463.2                                                                                                                     
[5]	valid_0's rmse: 442.992                                                                                                                   
[6]	valid_0's rmse: 429.224                                                                                                                   

[114]	valid_0's rmse: 147.835                                                                                                                 
[115]	valid_0's rmse: 147.978                                                                                                                 
[116]	valid_0's rmse: 148.123                                                                                                                 
[117]	valid_0's rmse: 148.166                                                                                                                 
[118]	valid_0's rmse: 147.942                                                                                                                 
[119]	valid_0's rmse: 147.556                                                                                                                 
[120]	valid_0's rmse: 147.636                                                                                                                 

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



...

[1]	valid_0's rmse: 517.965                                                                                                                   
Training until validation scores don't improve for 100 rounds                                                                                 
[2]	valid_0's rmse: 501.451                                                                                                                   
[3]	valid_0's rmse: 480.355                                                                                                                   
[4]	valid_0's rmse: 458.252                                                                                                                   
[5]	valid_0's rmse: 436.335                                                                                                                   
[6]	valid_0's rmse: 421.194                                                                                                                   

[114]	valid_0's rmse: 146.132                                                                                                                 
[115]	valid_0's rmse: 146.278                                                                                                                 
[116]	valid_0's rmse: 145.799                                                                                                                 
[117]	valid_0's rmse: 145.512                                                                                                                 
[118]	valid_0's rmse: 145.45                                                                                                                  
[119]	valid_0's rmse: 145.043                                                                                                                 
[120]	valid_0's rmse: 145.109                                                                                                                 

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[1]	valid_0's rmse: 519.526                                                                                                                   
Training until validation scores don't improve for 100 rounds                                                                                 
[2]	valid_0's rmse: 505.448                                                                                                                   
[3]	valid_0's rmse: 484.956                                                                                                                   
[4]	valid_0's rmse: 462.893                                                                                                                   
[5]	valid_0's rmse: 441.944                                                                                                                   
[6]	valid_0's rmse: 427.154                                                                                                                   

[114]	valid_0's rmse: 150.82                                                                                                                  
[115]	valid_0's rmse: 150.738                                                                                                                 
[116]	valid_0's rmse: 150.954                                                                                                                 
[117]	valid_0's rmse: 150.957                                                                                                                 
[118]	valid_0's rmse: 150.974                                                                                                                 
[119]	valid_0's rmse: 150.648                                                                                                                 
[120]	valid_0's rmse: 150.682                                                                                                                 

In [68]:
RMSE = lightgbm_factory(best)
print('best :', best)
print('best param after transform :')
argsDict_tranform(best,isPrint=True)
print('rmse of the best lightgbm:', np.sqrt(RMSE))

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's rmse: 517.855
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 500.855
[3]	valid_0's rmse: 480.889
[4]	valid_0's rmse: 458.929
[5]	valid_0's rmse: 436.313
[6]	valid_0's rmse: 421.287
[7]	valid_0's rmse: 406.908
[8]	valid_0's rmse: 389.283
[9]	valid_0's rmse: 371.905
[10]	valid_0's rmse: 351.755
[11]	valid_0's rmse: 340.865
[12]	valid_0's rmse: 326.71
[13]	valid_0's rmse: 312.683
[14]	valid_0's rmse: 304.256
[15]	valid_0's rmse: 292.125
[16]	valid_0's rmse: 279.826
[17]	valid_0's rmse: 270.001
[18]	valid_0's rmse: 262.883
[19]	valid_0's rmse: 253.339
[20]	valid_0's rmse: 244.888
[21]	valid_0's rmse: 236.108
[22]	valid_0's rmse: 228.934
[23]	valid_0's rmse: 221.423
[24]	valid_0's rmse: 213.452
[25]	valid_0's rmse: 206.99
[26]	valid_0's rmse: 203.479
[27]	valid_0's rmse: 201.441
[28]	valid_0's rmse: 196.428
[29]	valid_0's rmse: 192.386
[30]	valid_0's rmse: 187.847
[31]	valid_0's rmse: 184.221
[32]	valid_0's rmse: 180.793
[33]	valid_0's rmse: 

# 分月

In [10]:
# 分月
df_lgb = pd.DataFrame({'id': test['id']})
for col_add in ['ad_ry_mean', 'md_ry_mean', 'bt_ry_mean']:
    # 取用的字段，用于训练模型
    num_feat = shift_feat
    cate_feat = ['adcode', 'bodyType', 'model', 'regYear', 'regMonth', 'happyNY']
    features = num_feat + cate_feat + ['popularity', 'carCommentVolum', 'newsReplyVolum', 
                                       'n_popularity', 'n_carCommentVolum', 'n_newsReplyVolum', 'weightMonth'] + [col_add]  # [ad_ry_mean, md_ry_mean, bt_ry_mean]
 
    train_idx = (data['mt'] <= 20) # 小于等于20月以内的数据作为训练集
    valid_idx = (data['mt'].between(21, 24)) # 21到24个月的数据作为验证集
    test_idx = (data['mt'] > 24) # 大于24个月的是测试集
 
    # label
    data['n_label'] = np.log(data['label'])
 
    train_x = data[train_idx][features]
    train_y = data[train_idx]['n_label']
 
    valid_x = data[valid_idx][features]
    valid_y = data[valid_idx]['n_label']
 
    ############################### lgb ###################################
    lgb_model = lgb.LGBMRegressor(
        num_leaves=40, reg_alpha=1, reg_lambda=0.1, objective='mse',
        max_depth=-1, learning_rate=0.05, min_child_samples=5, random_state=2019,
        n_estimators=8000, subsample=0.8, colsample_bytree=0.8)
 
    lgb_model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)],
                  categorical_feature=cate_feat, early_stopping_rounds=100, verbose=300)
    data['pred_label'] = np.e ** lgb_model.predict(data[features])
    model = lgb_model
    # 特征重要程度
    print ('lgb特征重要程度：',sorted(dict(zip(train_x.columns,model.feature_importances_)).items(),key=lambda x: x[1], reverse=True))
    print('NRMSE的均值:',score(data = data[valid_idx]))
    model.n_estimators = model.best_iteration_
    model.fit(data[~test_idx][features], data[~test_idx]['n_label'], categorical_feature=cate_feat)
    data['forecastVolum'] = np.e ** model.predict(data[features])
    sub = data[test_idx][['id']]
    sub['forecastVolum'] = data[test_idx]['forecastVolum'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
    sub_lgb = sub.reset_index(drop=True)
    sub_lgb = sub_lgb[['id','forecastVolum']]
    print('lgb中forecastVolmn的0值数量：',(sub_lgb['forecastVolum']==0).sum())
    df_lgb[col_add] = sub_lgb['forecastVolum']

Training until validation scores don't improve for 100 rounds
[300]	valid_0's l2: 0.113983
[600]	valid_0's l2: 0.111103
[900]	valid_0's l2: 0.109772
[1200]	valid_0's l2: 0.108185
[1500]	valid_0's l2: 0.107045
[1800]	valid_0's l2: 0.106456
[2100]	valid_0's l2: 0.105942
[2400]	valid_0's l2: 0.105634
Early stopping, best iteration is:
[2373]	valid_0's l2: 0.105561
lgb特征重要程度： [('model', 16334), ('adcode', 11728), ('shift_model_adcode_mt_label_4', 11647), ('popularity', 10077), ('regMonth', 9658), ('carCommentVolum', 9112), ('newsReplyVolum', 8913), ('ad_ry_mean', 5056), ('n_popularity', 2071), ('n_newsReplyVolum', 1946), ('n_carCommentVolum', 1856), ('bodyType', 1495), ('regYear', 1399), ('happyNY', 972), ('weightMonth', 283)]
NRMSE的均值: 0.7384237387516983
lgb中forecastVolmn的0值数量： 0
Training until validation scores don't improve for 100 rounds
[300]	valid_0's l2: 0.116467
[600]	valid_0's l2: 0.113106
[900]	valid_0's l2: 0.111665
[1200]	valid_0's l2: 0.110511
[1500]	valid_0's l2: 0.109371
[18

# 鱼佬模型

In [11]:
def score(data, pred='pred_label', label='label', group='model'):
    data['pred_label'] = data['pred_label'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
    data_agg = data.groupby('model').agg({
        pred:  list,
        label: [list, 'mean']
    }).reset_index()
    data_agg.columns = ['_'.join(col).strip() for col in data_agg.columns]
    nrmse_score = []
    for raw in data_agg[['{0}_list'.format(pred), '{0}_list'.format(label), '{0}_mean'.format(label)]].values:
        nrmse_score.append(
            mse(raw[0], raw[1]) ** 0.5 / raw[2]
        )
    print(1 - np.mean(nrmse_score))
    return 1 - np.mean(nrmse_score)	

In [18]:
def get_model_type(train_x,train_y,valid_x,valid_y,m_type='lgb'):   
    if m_type == 'lgb':
        model = lgb.LGBMRegressor(
                                num_leaves=2**5-1, reg_alpha=0.25, reg_lambda=0.25, objective='mse',
                                max_depth=-1, learning_rate=0.05, min_child_samples=5, random_state=2019,
                                n_estimators=2000, subsample=0.9, colsample_bytree=0.7,
                                )
        model.fit(train_x, train_y, 
              eval_set=[(train_x, train_y),(valid_x, valid_y)],
              categorical_feature=cate_feat,
              early_stopping_rounds=100, verbose=100)      
    elif m_type == 'xgb':
        model = xgb.XGBRegressor(
                                max_depth=5 , learning_rate=0.05, n_estimators=2000, 
                                objective='reg:gamma', tree_method = 'hist',subsample=0.9, 
                                colsample_bytree=0.7, min_child_samples=5,eval_metric = 'rmse' 
                                )
        model.fit(train_x, train_y, 
              eval_set=[(train_x, train_y),(valid_x, valid_y)], 
              early_stopping_rounds=100, verbose=100)   
    return model

In [19]:
def get_train_model(df_, m, m_type='lgb'):
    df = df_.copy()
    # 数据集划分
    st = 1
    all_idx   = (df['mt'].between(st , m-1))
    train_idx = (df['mt'].between(st , m-5))
    valid_idx = (df['mt'].between(m-4, m-4))
    test_idx  = (df['mt'].between(m  , m  ))
    print('all_idx  :',st ,m-1)
    print('train_idx:',st ,m-5)
    print('valid_idx:',m-4,m-4)
    print('test_idx :',m  ,m  )  
    # 最终确认
    train_x = df[train_idx][features]
    train_y = df[train_idx]['label']
    valid_x = df[valid_idx][features]
    valid_y = df[valid_idx]['label']   
    # get model
    model = get_model_type(train_x,train_y,valid_x,valid_y,m_type)  
    # offline
    df['pred_label'] = model.predict(df[features])
    best_score = score(df[valid_idx]) 
    # online
    if m_type == 'lgb':
        model.n_estimators = model.best_iteration_ + 100
        model.fit(df[all_idx][features], df[all_idx]['label'], categorical_feature=cate_feat)
    elif m_type == 'xgb':
        model.n_estimators = model.best_iteration + 100
        model.fit(df[all_idx][features], df[all_idx]['label'])
    df['forecastVolum'] = model.predict(df[features]) 
    print('valid mean:',df[valid_idx]['pred_label'].mean())
    print('true  mean:',df[valid_idx]['label'].mean())
    print('test  mean:',df[test_idx]['forecastVolum'].mean())
    # 阶段结果
    sub = df[test_idx][['id']]
    sub['forecastVolum'] = df[test_idx]['forecastVolum'].apply(lambda x: 0 if x < 0 else x).round().astype(int)  
    return sub,df[valid_idx]['pred_label']

In [20]:
for month in [25,26,27,28]: 
    m_type = 'lgb' 
    
    data_df = data.copy()
    
    num_feat = shift_feat
    cate_feat = ['adcode', 'bodyType', 'model', 'regYear', 'regMonth', 'happyNY']
    features = num_feat + cate_feat + ['popularity', 'carCommentVolum', 'newsReplyVolum', 
                                       'n_popularity', 'n_carCommentVolum', 'n_newsReplyVolum', 'weightMonth']# + [col_add]  # [ad_ry_mean, md_ry_mean, bt_ry_mean]

    if m_type == 'lgb':
        for i in cate_feat:
            data_df[i] = data_df[i].astype('category')
    elif m_type == 'xgb':
        lbl = LabelEncoder()  
        for i in tqdm(cate_feat):
            data_df[i] = lbl.fit_transform(data_df[i].astype(str))
           
    features = num_feat + cate_feat
    print(len(features), len(set(features)))   
    
    sub,val_pred = get_train_model(data_df, month, m_type)   
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'salesVolume'] = sub['forecastVolum'].values
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'label'      ] = sub['forecastVolum'].values
sub = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
sub.columns = ['id','forecastVolum']
sub[['id','forecastVolum']].round().astype(int).to_csv('../rst/057_feature_yulao_model.csv', index=False)

9 8
all_idx  : 1 24
train_idx: 1 20
valid_idx: 21 21
test_idx : 25 25
Training until validation scores don't improve for 100 rounds
[100]	training's l2: 36159.2	valid_1's l2: 41382.6
[200]	training's l2: 22263.4	valid_1's l2: 36106.5
[300]	training's l2: 16997.1	valid_1's l2: 35685.7
[400]	training's l2: 14118.9	valid_1's l2: 35212.1
[500]	training's l2: 12106.3	valid_1's l2: 34785.3
[600]	training's l2: 10579.4	valid_1's l2: 34706.7
[700]	training's l2: 9432.25	valid_1's l2: 34328.5
[800]	training's l2: 8401.27	valid_1's l2: 34048.1
[900]	training's l2: 7592.19	valid_1's l2: 33787.8
[1000]	training's l2: 6903.27	valid_1's l2: 33794.6
Early stopping, best iteration is:
[914]	training's l2: 7511.57	valid_1's l2: 33660.8
0.7398800347208605
valid mean: 648.9083797665321
true  mean: 646.1667803030306
test  mean: 653.3836292495974
9 8
all_idx  : 1 25
train_idx: 1 21
valid_idx: 22 22
test_idx : 26 26
Training until validation scores don't improve for 100 rounds
[100]	training's l2: 36019.4	v

In [50]:
df_lgb.to_csv(path + "../rst/057_baseline_df_lgb_all_log_shift4_abnormal_95_9_05_1.csv", index=False) 
# df_lgb有三列值，任一一列提交，上0.57，祝各位好运！！

In [10]:
df_lgb.sample(10)

,id
3041,3086
1314,1337
1844,1867
4356,4423
4864,4931
3891,3958
1725,1748
2866,2911
5136,5225
896,897


In [11]:
data['rate'] = data.apply(lambda x: x['popularity'] / x['popularity%d'%i], axis=1)

KeyError: ('popularity4', 'occurred at index 0')

In [7]:
data.merge(tmp,on='model_adcode_mt').sample(10)

,ad_ry_mean,adcode,bodyType,bt_ry_mean,carCommentVolum,id,md_ry_mean,model,newsReplyVolum,popularity,regMonth,regYear,label,mt,model_adcode,model_adcode_mt,happyNY,model_adcode_mt_4,shift_model_adcode_mt_label_4,popularity_4_x,weightMonth,n_carCommentVolum,n_popularity,n_newsReplyVolum,popularity_4_y
27988,623.000000,510000,0,459.000000,2967.0,1611,938.000000,12,8801.0,5994.0,2,2018,NaN,26,510012,51001226,1,51001230,1043.0,1921.0,6,7.995307,8.698514,9.082621,1921.0
6422,413.245833,610000,0,544.339848,80.0,0,415.670455,51,1317.0,2459.0,9,2016,293.0,9,610051,61005109,0,61005113,263.0,2154.0,4,4.382027,7.807510,7.183112,2154.0
3733,552.731944,420000,3,453.318182,236.0,0,428.791667,49,2023.0,1321.0,7,2016,280.0,7,420049,42004907,0,42004911,391.0,1284.0,4,5.463832,7.186144,7.612337,1284.0
14833,521.523611,340000,1,672.187991,231.0,0,640.386364,14,478.0,847.0,4,2017,584.0,16,340014,34001416,0,34001420,1070.0,665.0,6,5.442418,6.741701,6.169611,665.0
28581,215.000000,110000,0,459.000000,583.0,2204,584.000000,39,13369.0,5863.0,2,2018,NaN,26,110039,11003926,1,11003930,121.0,1251.0,6,6.368187,8.676417,9.500694,1251.0
2435,552.731944,420000,1,711.211841,79.0,0,400.340909,50,1540.0,1945.0,6,2016,257.0,6,420050,42005006,0,42005010,189.0,4361.0,4,4.369448,7.573017,7.339538,4361.0
9663,571.986111,340000,0,544.339848,214.0,0,873.386364,19,1523.5,2467.0,12,2016,1490.0,12,340019,34001912,0,34001916,980.0,2050.0,4,5.365976,7.810758,7.328766,2050.0
27561,364.000000,350000,1,654.000000,442.0,1184,363.000000,52,0.0,1823.0,1,2018,NaN,25,350052,35005225,0,35005229,522.0,1483.0,6,6.091310,7.508239,-inf,1483.0
27247,351.000000,360000,0,459.000000,784.0,848,667.000000,38,10111.0,17.0,1,2018,NaN,25,360038,36003825,0,36003829,402.0,95.0,6,6.664409,2.833213,9.221379,95.0
31498,478.000000,430000,0,459.000000,841.0,5187,314.000000,51,613.0,1833.0,4,2018,NaN,28,430051,43005128,0,43005132,357.0,1515.0,6,6.734592,7.513709,6.418365,1515.0


In [ ]:
df_lgb = pd.DataFrame({'id': test['id']})
for col_add in ['ad_ry_mean', 'md_ry_mean', 'bt_ry_mean']:
    # 取用的字段，用于训练模型
    num_feat = shift_feat
    cate_feat = ['adcode', 'bodyType', 'model', 'regYear', 'regMonth', 'happyNY']
    features = num_feat + cate_feat + ['popularity', 'carCommentVolum', 'newsReplyVolum', 
                                       'n_popularity', 'n_carCommentVolum', 'n_newsReplyVolum', 'weightMonth'] + [col_add]  # [ad_ry_mean, md_ry_mean, bt_ry_mean]
 
    train_idx = (data['mt'] <= 20) # 小于等于20月以内的数据作为训练集
    valid_idx = (data['mt'].between(21, 24)) # 21到24个月的数据作为验证集
    test_idx = (data['mt'] > 24) # 大于24个月的是测试集
 
    # label
    data['n_label'] = np.log(data['label'])
 
    train_x = data[train_idx][features]
    train_y = data[train_idx]['n_label']
 
    valid_x = data[valid_idx][features]
    valid_y = data[valid_idx]['n_label']
 
    ############################### lgb ###################################
    lgb_model = lgb.LGBMRegressor(
        num_leaves=40, reg_alpha=1, reg_lambda=0.1, objective='mse',
        max_depth=-1, learning_rate=0.05, min_child_samples=5, random_state=2019,
        n_estimators=8000, subsample=0.8, colsample_bytree=0.8)
 
    lgb_model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)],
                  categorical_feature=cate_feat, early_stopping_rounds=100, verbose=300)
    data['pred_label'] = np.e ** lgb_model.predict(data[features])
    model = lgb_model
    # 特征重要程度
    print ('lgb特征重要程度：',sorted(dict(zip(train_x.columns,model.feature_importances_)).items(),key=lambda x: x[1], reverse=True))
    print('NRMSE的均值:',score(data = data[valid_idx]))
    model.n_estimators = model.best_iteration_
    model.fit(data[~test_idx][features], data[~test_idx]['n_label'], categorical_feature=cate_feat)
    data['forecastVolum'] = np.e ** model.predict(data[features])
    sub = data[test_idx][['id']]
    sub['forecastVolum'] = data[test_idx]['forecastVolum'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
    sub_lgb = sub.reset_index(drop=True)
    sub_lgb = sub_lgb[['id','forecastVolum']]
    print('lgb中forecastVolmn的0值数量：',(sub_lgb['forecastVolum']==0).sum())
    df_lgb[col_add] = sub_lgb['forecastVolum']